In [3]:
import pandas as pd
import numpy as np
import requests
import time
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas import datetime
import datedelta
import plotly
plotly.tools.set_credentials_file(username='sasi_ynwa', api_key='wEm5TtVT8TOVKzFyd8CA')
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [4]:
r = requests.get(url='https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&end=9999999999&period=14400&start=1405699200')

In [5]:
dfBTC = pd.DataFrame.from_records(r.json())

In [6]:
start =time.time()#works but first create a column wiht DateTime
dfBTC['DateTime'] = pd.Series(np.random.randn(len(dfBTC)), index=dfBTC.index)
for i, row in dfBTC.iterrows():
    dfBTC['DateTime'][i] = datetime.fromtimestamp(row['date']).strftime("%m-%d-%Y %H:%M:%S")
dfBTC['DateTime'] =  pd.to_datetime(dfBTC['DateTime'], format='%m-%d-%Y %H:%M:%S')
end = time.time()
print(end - start)

C:\Users\msasi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\msasi\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



314.55490159988403


In [7]:
#Using Roller to generate Moving Averages(Simple). Here we re multiplying by 6 cos each day has 6 recordings
roller = dfBTC['close'].rolling(10*6)
dfBTC['10DayMA'] = roller.mean()

roller = dfBTC['close'].rolling(30*6)
dfBTC['30DayMA'] = roller.mean()

roller = dfBTC['close'].rolling(50*6)
dfBTC['50DayMA'] = roller.mean()

In [9]:
data = [
    go.Scatter(
        x=dfBTC['DateTime'], # assign x as the dataframe column 'x'
        y=dfBTC['close'],
        name = 'Closing Price'
    ),
    go.Scatter(
        x = dfBTC['DateTime'],
        y = dfBTC['10DayMA'],
        name = '10DayMA'
    ),
   # go.Scatter(
    #    x = dfBTC['DateTime'],
     #   y = dfBTC['30DayMA'],
      #  name = '30DayMA'
    #),
    #go.Scatter(
     #   x = dfBTC['DateTime'],
      #  y = dfBTC['50DayMA'],
       # name = '50DayMA'
    #),
    go.Candlestick(x=dfBTC.DateTime,
                       open=dfBTC.open,
                       high=dfBTC.high,
                       low=dfBTC.low,
                       close=dfBTC.close,
                       increasing=dict(line=dict(color= '#17BECF')),
                       decreasing=dict(line=dict(color= '#7F7F7F')))
]
# IPython notebook
plotly.offline.iplot(data)

#url = py.plot(data, filename='pandas/basic-line-plot')


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [7]:
#Using Roller to generate Moving Averages(Simple) for Volume
roller = dfBTC['volume'].rolling(10*6)
dfBTC['Vol10DayMA'] = roller.mean()

roller = dfBTC['volume'].rolling(30*6)
dfBTC['Vol30DayMA'] = roller.mean()

roller = dfBTC['volume'].rolling(50*6)
dfBTC['Vol50DayMA'] = roller.mean()

In [8]:
data = [
    go.Scatter(
        x=dfBTC['DateTime'], # assign x as the dataframe column 'x'
        y=dfBTC['volume'],
        name = 'Volume'
    ),
    go.Scatter(
        x = dfBTC['DateTime'],
        y = dfBTC['Vol50DayMA'],
        name = '50DayMA - Volume'
    ),
    go.Scatter(
        x = dfBTC['DateTime'],
        y = dfBTC['Vol30DayMA'],
        name = '30DayMA - Volume'
    ),
    go.Scatter(
        x = dfBTC['DateTime'],
        y = dfBTC['Vol10DayMA'],
        name = '10DayMA - Volume'
    )#,
    #go.Candlestick(x=dfETH.DateTime,
     #                  open=dfETH.open,
      #                 high=dfETH.high,
       #                low=dfETH.low,
        #               close=dfETH.close,
         #              increasing=dict(line=dict(color= '#17BECF')),
          #             decreasing=dict(line=dict(color= '#7F7F7F')))
]
# IPython notebook
plotly.offline.iplot(data)

In [11]:
#New try, Volatility window is for 3 days shift should be done -1 since we are comparing results with 1 up 

dfBTC['Returns'] = ((dfBTC['close'] / dfBTC['close'].shift(-1)) - 1)
dfBTC['Volatility'] = pd.rolling_std(dfBTC['Returns'], window=18)
#Using Roller to generate Moving Averages(Simple) for Volatility
roller = dfBTC['Volatility'].rolling(10*6)
dfBTC['Volatility10DayMA'] = roller.mean()

roller = dfBTC['Volatility'].rolling(30*6)
dfBTC['Volatility30DayMA'] = roller.mean()

roller = dfBTC['Volatility'].rolling(50*6)
dfBTC['Volatility50DayMA'] = roller.mean()

C:\Users\msasi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning:

pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=18,center=False).std()



In [12]:
dfBTC['meanLineVolatility'] = dfBTC['Volatility'].mean()

In [13]:
data = [
    go.Scatter(
        x=dfBTC['DateTime'], # assign x as the dataframe column 'x'
        y=dfBTC['Volatility'],
        name = 'Volatility'
    ),
    go.Scatter(
        x=dfBTC['DateTime'],
        y=dfBTC['meanLineVolatility'],
        name = 'Volatility mean Line'
    ),
    go.Scatter(
        x = dfBTC['DateTime'],
        y = dfBTC['Volatility10DayMA'],
        name = '10DayMA - Volatility'
    ),
    go.Scatter(
        x = dfBTC['DateTime'],
        y = dfBTC['Volatility30DayMA'],
        name = '30DayMA - Volatility'
    ),
    go.Scatter(
        x = dfBTC['DateTime'],
        y = dfBTC['Volatility50DayMA'],
        name = '50DayMA - Volatility'
    )#,
    #go.Candlestick(x=dfETH.DateTime,
     #                  open=dfETH.open,
      #                 high=dfETH.high,
       #                low=dfETH.low,
        #               close=dfETH.close,
         #              increasing=dict(line=dict(color= '#17BECF')),
          #             decreasing=dict(line=dict(color= '#7F7F7F')))
]
# IPython notebook
plotly.offline.iplot(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [65]:
from itertools import islice

def window(seq, n=50):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [66]:
def moving_averages(values, size):
    for selection in window(values, size):
        yield sum(selection) / size

In [84]:
x=0
y=0
for avg in moving_averages(dfBTC['close'], 50):
    #print(avg)
    y=50+x
    dfBTC.iloc[y,dfBTC.columns.get_loc('50DayMA')] = avg
    x = x+1


IndexError: single positional indexer is out-of-bounds

In [79]:
from itertools import islice

def window(seq, n=200):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result
        

In [80]:
def moving_averages(values, size):
    for selection in window(values, size):
        yield sum(selection) / size

In [81]:
x=0
y=0
for avg in moving_averages(dfBTC['close'], 200):
    #print(avg)
    y=200+x
    dfBTC.iloc[y,dfBTC.columns.get_loc('200DayMA')] = avg
    x = x+1

IndexError: single positional indexer is out-of-bounds

In [82]:
dfBTC.head()

,close,date,high,low,open,quoteVolume,volume,weightedAverage,DateTime,50DayMA,200DayMA
0,244.00,1424304000,244.000,225.00,225.000000,0.193117,46.276313,239.627778,2015-02-18 18:00:00,-0.120884,-0.141968
1,240.25,1424390400,245.000,240.25,240.250118,0.230429,55.894897,242.568479,2015-02-19 18:00:00,-1.293238,1.560839
2,245.00,1424476800,245.000,245.00,245.000000,0.060091,14.722239,245.000000,2015-02-20 18:00:00,0.219581,0.103464
3,235.00,1424563200,249.000,235.00,245.000000,0.539055,129.121248,239.532608,2015-02-21 18:00:00,0.174718,-0.353447
4,235.00,1424649600,235.001,235.00,235.000002,0.410926,96.567562,235.000062,2015-02-22 18:00:00,-0.545162,0.419150
